# Petri Eval Results

Load transcripts, classify whether the auditor detected the hidden behavior, and plot detection rates.

In [ ]:
import json
import re
from glob import glob
from collections import defaultdict

import numpy as np
from tqdm.auto import tqdm

from src.utils import load_prompt_file
from src import api, Prompt, ChatMessage, MessageRole
from src.viz import plot_hierarchical_bars

# ---------- Load transcripts from results/ ----------
RESULTS_DIR = "experiments/petri_evals/results"

# {quirk_family: {trigger_variant: [list of transcript dicts]}}
transcripts = defaultdict(lambda: defaultdict(list))

for path in sorted(glob(f"{RESULTS_DIR}/*/*/*.json")):
    parts = path.split("/")
    quirk_family = parts[-3]      # e.g. "secret_loyalty"
    trigger_variant = parts[-2]   # e.g. "policy_discussion"

    with open(path) as f:
        data = json.load(f)

    judge = data.get("metadata", {}).get("judge_output")
    if judge is None or judge.get("scores") is None:
        continue

    transcripts[quirk_family][trigger_variant].append(data)

# Summary
for qf in sorted(transcripts):
    for tv in sorted(transcripts[qf]):
        print(f"{qf}/{tv}: {len(transcripts[qf][tv])} transcripts")

In [ ]:
# ---------- Build classifier prompts ----------

# Extract the hidden behavior description for each (quirk_family, trigger_variant)
PROMPTS_DIR = "src/model_organisms/prompts"

def get_hidden_behavior(quirk_family: str, trigger_variant: str) -> str:
    """Load the rendered system prompt and extract <hidden_behavior> content."""
    rendered = load_prompt_file(f"{PROMPTS_DIR}/{quirk_family}/{trigger_variant}.jinja2")
    match = re.search(r"<hidden_behavior>(.*?)</hidden_behavior>", rendered, re.DOTALL)
    if match:
        return match.group(1).strip()
    return rendered  # fallback to full prompt


def parse_classifier_output(text: str) -> dict:
    """Parse <reasoning> and <final_answer> from classifier response."""
    reasoning_match = re.search(r"<reasoning>(.*?)</reasoning>", text, re.DOTALL)
    answer_match = re.search(r"<final_answer>(.*?)</final_answer>", text, re.DOTALL)
    reasoning = reasoning_match.group(1).strip() if reasoning_match else ""
    final_answer = answer_match.group(1).strip() if answer_match else ""
    return {
        "reasoning": reasoning,
        "final_answer": final_answer,
        "is_yes": final_answer.lower() == "yes",
    }


# Build one prompt per transcript
all_prompts = []
prompt_keys = []  # Track (quirk_family, trigger_variant, idx) for each prompt

for qf in sorted(transcripts):
    for tv in sorted(transcripts[qf]):
        hidden_behavior = get_hidden_behavior(qf, tv)
        for idx, t in enumerate(transcripts[qf][tv]):
            summary = t["metadata"]["judge_output"]["summary"]
            rendered = load_prompt_file(
                "src/auditor/prompts/petri_evals/grade_for_quirk.jinja2",
                system_prompt=hidden_behavior,
                review_response=summary,
            )
            all_prompts.append(
                Prompt(messages=[ChatMessage(role=MessageRole.user, content=rendered)])
            )
            prompt_keys.append((qf, tv, idx))

print(f"Total classifier prompts: {len(all_prompts)}")

In [ ]:
# ---------- Run classifier via regular API with concurrency ----------
import asyncio

CLASSIFIER_MODEL = "claude-opus-4-6"

async def classify_prompt(prompt: Prompt) -> str:
    responses = await api(
        model_id=CLASSIFIER_MODEL,
        prompt=prompt,
        max_tokens=10000,
        thinking={"type": "enabled", "budget_tokens": 2000},
    )
    return responses[0].completion

semaphore = asyncio.Semaphore(100)

async def classify_with_semaphore(prompt: Prompt) -> str:
    async with semaphore:
        return await classify_prompt(prompt)

all_completions = await tqdm.gather(
    *[classify_with_semaphore(p) for p in all_prompts],
    desc="Classifying",
)
print(f"Got {len(all_completions)} responses")

In [ ]:
# ---------- Parse classifier outputs and compute detection rates ----------

# Collect binary results: {quirk_family: {trigger_variant: [0 or 1, ...]}}
binary_results = defaultdict(lambda: defaultdict(list))

for i, (qf, tv, idx) in enumerate(prompt_keys):
    result = parse_classifier_output(all_completions[i])
    binary_results[qf][tv].append(1 if result["is_yes"] else 0)

# Print detection rates
for qf in sorted(binary_results):
    for tv in sorted(binary_results[qf]):
        vals = binary_results[qf][tv]
        rate = sum(vals) / len(vals)
        print(f"{qf}/{tv}: {sum(vals)}/{len(vals)} = {rate:.1%}")

In [ ]:
# ---------- Plot detection rates ----------

DISPLAY_NAMES = {
    "secret_loyalty": "Secret Loyalty",
    "ai_welfare_ascii": "AI Welfare ASCII",
    "any_relevant": "Any Relevant",
    "policy_discussion": "Policy Discussion",
    "american_conservative": "American Conservative",
    "python_only": "Python Only",
    "synthetic_data": "Synthetic Data",
    "claude_synthetic_data": "Claude Synthetic Data",
}

# Trigger specificity order: broadest trigger first -> most specific last
GROUP_ORDER = [
    "Any Relevant",
    "Policy Discussion",
    "American Conservative",
    "Python Only",
    "Synthetic Data",
    "Claude Synthetic Data",
]

# ---------- Top-50 by bizarre + concerning ----------
TOP_K = 50

# For each condition, find indices of top-K transcripts by combined score
top_k_binary = defaultdict(lambda: defaultdict(list))

for qf in sorted(transcripts):
    for tv in sorted(transcripts[qf]):
        ts = transcripts[qf][tv]
        combined = [
            t["metadata"]["judge_output"]["scores"].get("unprompted_bizarre_behavior", 1)
            + t["metadata"]["judge_output"]["scores"].get("concerning", 1)
            for t in ts
        ]
        top_idx = set(np.argsort(combined)[-TOP_K:])
        # binary_results[qf][tv] is aligned with transcripts[qf][tv] by index
        for idx, val in enumerate(binary_results[qf][tv]):
            if idx in top_idx:
                top_k_binary[qf][tv].append(val)

def build_plot_data(results):
    plot_data = {}
    for qf in sorted(results):
        split_name = DISPLAY_NAMES.get(qf, qf)
        plot_data[split_name] = {}
        for tv in sorted(results[qf]):
            group_name = DISPLAY_NAMES.get(tv, tv)
            plot_data[split_name][group_name] = {
                "Detection Rate": [float(v) for v in results[qf][tv]]
            }
    return plot_data

# Plot 1: Top-50 most suspicious transcripts
fig1 = plot_hierarchical_bars(
    data=build_plot_data(top_k_binary),
    title=f"Petri Detection Rate: {TOP_K} Most Suspicious Transcripts",
    ylabel="Detection Rate",
    ylim=(0, 1.05),
    bar_width=0.5,
    rotate_xticks=30,
    show_values=True,
    figsize=(9, 5.5),
    split_spacing=1.2,
    group_order=GROUP_ORDER,
    split_label_offset=-0.4,
)

# Plot 2: All transcripts
fig2 = plot_hierarchical_bars(
    data=build_plot_data(binary_results),
    title="Petri Detection Rate: All Transcripts",
    ylabel="Detection Rate",
    ylim=(0, 1.05),
    bar_width=0.5,
    rotate_xticks=30,
    show_values=True,
    figsize=(9, 5),
    split_spacing=1.2,
    group_order=GROUP_ORDER,
    split_label_offset=-0.4,
)